# Quantized Model Synthesis
This notebook will follow up from the end of the cleanup notebook to convert the model to HLS layers all the way up to synthesis of the model using Vivado

### Conversion to HLS Layers

In [1]:
import onnx
from finn.util.test import get_test_model_trained
import brevitas.onnx as bo
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.fold_constants import FoldConstants
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.transformation.infer_datatypes import InferDataTypes
from finn.util.visualization import showInNetron
from finn.transformation.infer_data_layouts import InferDataLayouts

In [3]:
from inspect import getmembers, isclass

import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
print(getmembers(to_hls, isclass))

[('DataType', <enum 'DataType'>), ('InferAddStreamsLayer', <class 'finn.transformation.fpgadataflow.convert_to_hls_layers.InferAddStreamsLayer'>), ('InferBinaryStreamingFCLayer', <class 'finn.transformation.fpgadataflow.convert_to_hls_layers.InferBinaryStreamingFCLayer'>), ('InferChannelwiseLinearLayer', <class 'finn.transformation.fpgadataflow.convert_to_hls_layers.InferChannelwiseLinearLayer'>), ('InferConvInpGen', <class 'finn.transformation.fpgadataflow.convert_to_hls_layers.InferConvInpGen'>), ('InferDataTypes', <class 'finn.transformation.infer_datatypes.InferDataTypes'>), ('InferDuplicateStreamsLayer', <class 'finn.transformation.fpgadataflow.convert_to_hls_layers.InferDuplicateStreamsLayer'>), ('InferGlobalAccPoolLayer', <class 'finn.transformation.fpgadataflow.convert_to_hls_layers.InferGlobalAccPoolLayer'>), ('InferLabelSelectLayer', <class 'finn.transformation.fpgadataflow.convert_to_hls_layers.InferLabelSelectLayer'>), ('InferPool_Batch', <class 'finn.transformation.fpgadat

In [5]:
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
model = ModelWrapper("quant_models/facile_streamlined.onnx")
mem_mode = "decoupled"

#model = model.transform(to_hls.InferBinaryStreamingFCLayer(mem_mode))
model = model.transform(to_hls.InferThresholdingLayer())                 #TODO: For some reason, these transforms
model = model.transform(to_hls.InferQuantizedStreamingFCLayer(mem_mode)) #aren't applying, maybe same reason as
#model = model.transform(to_hls.InferThresholdingLayer())                #MoveScalarAddPastMatMul?
#model = model.transform(InferDataLayouts())

model.save("quant_models/facile_hls_layers.onnx")
showInNetron("quant_models/facile_hls_layers.onnx")


Stopping http://0.0.0.0:8081
Serving 'quant_models/facile_hls_layers.onnx' at http://0.0.0.0:8081


### Dataflow Partitioning

In [5]:
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition

model = ModelWrapper("quant_models/facile_hls_layers.onnx")
parent_model = model.transform(CreateDataflowPartition())
parent_model.save("quant_models/facile_dataflow_parent.onnx")
showInNetron("quant_models/facile_dataflow_parent.onnx")


Stopping http://0.0.0.0:8081
Serving 'quant_models/facile_dataflow_parent.onnx' at http://0.0.0.0:8081


In [7]:
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")
#sdp_node = getCustomOp(sdp_node)
print(sdp_node)

[]
